# <a id='toc1_'></a>[GENIE Pre Variant Analysis](#toc0_)

This data uses `data_mutations_extended.txt` from Synapse. You will need to create an account to download data from [here](https://www.synapse.org/Synapse:syn68719440). This notebook expects the `data_mutations_extended.txt` to be in the same directory.

This notebook is used to run GENIE variant data through the variation-normalizer.

**Table of contents**<a id='toc0_'></a>    
- [GENIE Pre Variant Analysis](#toc1_)    
  - [Create dataframe](#toc1_1_)    
  - [Try using /normalize](#toc1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# Load environment variables. This MUST be the first cell.
from dotenv import load_dotenv

load_dotenv("../../../.env.shared")

True

In [2]:
import logging
import csv
from pathlib import Path

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from variation.query import QueryHandler
from variation.schemas.service_schema import ClinVarAssembly
from tqdm import tqdm


log_filename = "genie-pre-variant-analysis.log"
logging.basicConfig(
    filename=log_filename,
    format="[%(asctime)s] - %(name)s - %(levelname)s : %(message)s",
)

/Users/rsjxa001/variation-normalizer-manuscript/.venv/lib/python3.13/site-packages/hgvs/__init__.py:60: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# Download cool-seq-tool files. This must be done before initializing QueryHandler
import os
import sys

module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
from analysis.download_cool_seq_tool_files import download_cool_seq_tool_files  # noqa: E402

download_cool_seq_tool_files(is_docker_env=False)

## <a id='toc1_1_'></a>[Create dataframe](#toc0_)

In [4]:
# Create output directory
path = Path("variation_normalizer_output")
path.mkdir(exist_ok=True)

In [5]:
# Get GENIE Variant data
genie_variants_df = pd.read_csv(
    "data_mutations_extended.txt",
    sep="\t",
    usecols=[
        "Hugo_Symbol",
        "NCBI_Build",
        "Chromosome",
        "Start_Position",
        "End_Position",
        "Tumor_Sample_Barcode",
        "Reference_Allele",
        "Tumor_Seq_Allele2",
        "HGVSp_Short",
        "dbSNP_RS",
    ],
)
# Some positions are '-', we drop nan below
genie_variants_df = genie_variants_df.replace({"-": np.nan})

# Create free text (hgvs like) query for protein representation: f"{gene}: "{p_change}"
genie_variants_df["free_text_p_short"] = np.where(
    ~genie_variants_df["Hugo_Symbol"].isna() & ~genie_variants_df["HGVSp_Short"].isna(),
    genie_variants_df["Hugo_Symbol"] + " " + genie_variants_df["HGVSp_Short"],
    np.nan,
)

# Create gnomad vcf like query for genomic representation: f"{chromosome}-{pos}-{ref}-{alt}"
genie_variants_df["coordinates"] = np.where(
    ~genie_variants_df["Chromosome"].isna()
    & ~genie_variants_df["Start_Position"].isna()
    & ~genie_variants_df["End_Position"].isna()
    & ~genie_variants_df["Reference_Allele"].isna()
    & ~genie_variants_df["Tumor_Seq_Allele2"].isna()
    & ~genie_variants_df["NCBI_Build"].isna(),
    genie_variants_df["Chromosome"].astype(str)
    + "-"
    + genie_variants_df["Start_Position"].astype(str)
    + "-"
    + genie_variants_df["Reference_Allele"]
    + "-"
    + genie_variants_df["Tumor_Seq_Allele2"],
    np.nan,
)

# Create genie df csv
genie_variants_df.to_csv(
    "variation_normalizer_output/genie_variants_df.tsv", sep="\t", index=False
)

genie_variants_df.shape

/var/folders/_d/tfjn_dfn6wl35y6ywjvyxlhh0000gq/T/ipykernel_81508/2987082493.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  genie_variants_df = pd.read_csv(


(2738934, 12)

Protein (free text)

In [6]:
# Remove na values
free_text_df = genie_variants_df.copy()
free_text_df = free_text_df[free_text_df["free_text_p_short"].notna()]
free_text_df.shape

(2686363, 12)

In [7]:
# Drop duplicates
free_text_df = free_text_df.drop_duplicates(subset=["free_text_p_short"])
free_text_df.shape

(954230, 12)

In [8]:
# Create free text df csv (if we need to debug)
free_text_df.to_csv(
    "variation_normalizer_output/free_text_df.tsv", sep="\t", index=False
)

In [9]:
# Create list of possible protein free text queries
free_text_queries = [v for v in free_text_df["free_text_p_short"].values]

Genomic (coordinates)

In [10]:
# Remove na values
coordinates_df = genie_variants_df.copy()
coordinates_df = coordinates_df[coordinates_df["coordinates"].notna()]
coordinates_df.shape

(2397877, 12)

In [11]:
# Drop duplicates
coordinates_df = coordinates_df.drop_duplicates(subset=["coordinates"])
coordinates_df.shape

(963850, 12)

In [12]:
# Create coordinates df csv (if we need to debug)
coordinates_df.to_csv(
    "variation_normalizer_output/coordinates_df.tsv", sep="\t", index=False
)

In [13]:
# Create list of possible genomic gnomad vcf queries
coordinates_queries = [v for v in coordinates_df["coordinates"].values]

## <a id='toc1_2_'></a>[Try using /normalize](#toc0_)

In this section, we will run the queries through the variation normalizer.

In [14]:
query_handler = QueryHandler()

***Using Gene Database Endpoint: http://localhost:8000***


In [15]:
def translate_from_genomic(genomic_query: str) -> dict:
    """Try using vrs-python translate from using genomic query that failed to normalize"""
    resp = {"vrs_id": None, "error": None}
    try:
        translate_from_resp = query_handler.vrs_python_tlr.translate_from(
            genomic_query, assembly_name="GRCh37"
        )
    except Exception as e:
        resp["error"] = str(e)
    else:
        resp["vrs_id"] = translate_from_resp.id

    return resp

In [16]:
async def normalize_genie(queries: list[str], query_type: str):
    # This file contains GENIE Variant queries that we were not able to normalize.
    unable_to_normalize_wf = open(
        f"variation_normalizer_output/unable_to_normalize_{query_type}_queries.tsv", "w"
    )
    unable_to_normalize_wr = csv.writer(unable_to_normalize_wf, delimiter="\t")
    unable_to_normalize_wr.writerow(
        ["query", "exception_raised", "message", "warnings"]
    )

    # This file contains GENIE Variant queries that we were able to normalize.
    able_to_normalize_wf = open(
        f"variation_normalizer_output/able_to_normalize_{query_type}_queries.tsv", "w"
    )
    able_to_normalize_wr = csv.writer(able_to_normalize_wf, delimiter="\t")
    able_to_normalize_wr.writerow(["query", "vrs_id", "succeeded_endpoint"])

    for query in tqdm(queries):
        try:
            variation_norm_resp = await query_handler.normalize_handler.normalize(
                query, input_assembly=ClinVarAssembly.GRCH37
            )
        except Exception as e:
            warnings = [str(e)]

            # If genomic, try not lifting over to see if we can normalizer
            if query_type == "genomic":
                genomic_resp = translate_from_genomic(query)

                if genomic_resp["vrs_id"]:
                    vrs_id = genomic_resp["vrs_id"]
                else:
                    vrs_id = None
                    warnings.append(genomic_resp["error"])

                if vrs_id:
                    able_to_normalize_wr.writerow([query, vrs_id, "translate_from"])
                    continue

            unable_to_normalize_wr.writerow([query, True, str(e), None])
        else:
            if variation_norm_resp.variation:
                vrs_id = variation_norm_resp.variation.id
                able_to_normalize_wr.writerow([query, vrs_id, "normalize"])
            else:
                warnings = sorted(variation_norm_resp.warnings)

                # If genomic, try not lifting over to see if we can normalizer
                if query_type == "genomic":
                    genomic_resp = translate_from_genomic(query)

                    if genomic_resp["vrs_id"]:
                        vrs_id = genomic_resp["vrs_id"]
                    else:
                        vrs_id = None
                        warnings.append(genomic_resp["error"])

                    if vrs_id:
                        able_to_normalize_wr.writerow([query, vrs_id, "translate_from"])
                        continue

                unable_to_normalize_wr.writerow(
                    [query, False, "unable to normalize", warnings]
                )

    # Close all files
    unable_to_normalize_wf.close()
    able_to_normalize_wf.close()

In [17]:
await normalize_genie(free_text_queries, "protein")

100%|██████████| 954230/954230 [60:04:47<00:00,  4.41it/s]     


In [18]:
await normalize_genie(coordinates_queries, "genomic")

100%|██████████| 963850/963850 [10:17:52<00:00, 26.00it/s]  
